In [ ]:
!pip install protobuf==3.20.0 llama-index-vector-stores-chroma


In [ ]:
!pip install llama-index llama-index-packs-raptor llama-index-vector-stores-qdrant

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.43.post1
   

In [ ]:
from llama_index.packs.raptor import RaptorPack

In [ ]:
!wget https://arxiv.org/pdf/2401.18059.pdf -O ./raptor_paper.pdf

--2024-06-09 05:30:24--  https://arxiv.org/pdf/2401.18059.pdf
Resolving arxiv.org (arxiv.org)... 151.101.195.42, 151.101.67.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.195.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2401.18059 [following]
--2024-06-09 05:30:24--  http://arxiv.org/pdf/2401.18059
Connecting to arxiv.org (arxiv.org)|151.101.195.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2547113 (2.4M) [application/pdf]
Saving to: ‘./raptor_paper.pdf’

./raptor_paper.pdf  100%[===================>]   2.43M  --.-KB/s    in 0.1s    

2024-06-09 05:30:24 (23.4 MB/s) - ‘./raptor_paper.pdf’ saved [2547113/2547113]



In [ ]:
import os

os.environ["OPENAI_API_KEY"] ="..."

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["./raptor_paper.pdf"]).load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import chromadb

client = chromadb.PersistentClient(path="./raptor_paper_db")
collection = client.get_or_create_collection("raptor")

vector_store = ChromaVectorStore(chroma_collection=collection)

raptor_pack = RaptorPack(
    documents,
    embed_model=OpenAIEmbedding(
        model="text-embedding-3-small"
    ), # used for embedding clusters
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1), # used for generating summaries
    vector_store=vector_store, # used for storage
    similarity_top_k=2, # top k for each layer, or overall top-k for collapsed
    mode="collapsed", # sets default mode
    transformations=[
        SentenceSplitter(chunk_size=400, chunk_overlap=50)
    ], # transformations applied for ingestion
)

Generating embeddings for level 0.
Performing clustering for level 0.
Generating summaries for level 0 with 13 clusters.
Level 0 created summaries/clusters: 13
Generating embeddings for level 1.
Performing clustering for level 1.
Generating summaries for level 1 with 2 clusters.
Level 1 created summaries/clusters: 2
Generating embeddings for level 2.
Performing clustering for level 2.
Generating summaries for level 2 with 1 clusters.
Level 2 created summaries/clusters: 1


In [ ]:
# just to see if we can get the relevent chunks or not
nodes = raptor_pack.run("What baselines is raptor compared against?", mode="collapsed")
print(len(nodes))
print(nodes[0].text)

2
Specifically, RAPTOR’s F-1 scores are at least 1.8% points higher than DPR and at least 5.3% points
higher than BM25.
Retriever GPT-3 F-1 Match GPT-4 F-1 Match UnifiedQA F-1 Match
Title + Abstract 25.2 22.2 17.5
BM25 46.6 50.2 26.4
DPR 51.3 53.0 32.1
RAPTOR 53.1 55.7 36.6
Table 4: Comparison of accuracies on the QuAL-
ITY dev dataset for two different language mod-
els (GPT-3, UnifiedQA 3B) using various retrieval
methods. RAPTOR outperforms the baselines of
BM25 and DPR by at least 2.0% in accuracy.
Model GPT-3 Acc. UnifiedQA Acc.
BM25 57.3 49.9
DPR 60.4 53.9
RAPTOR 62.4 56.6
Table 5: Results on F-1 Match scores of various
models on the QASPER dataset.
Model F-1 Match
LongT5 XL (Guo et al., 2022) 53.1
CoLT5 XL (Ainslie et al., 2023) 53.9
RAPTOR + GPT-4 55.7Comparison to State-of-the-art Systems
Building upon our controlled comparisons,
we examine RAPTOR’s performance relative
to other state-of-the-art models.


In [ ]:
# just to see if we can get the relevent chunks or not
nodes = raptor_pack.run("What baselines is raptor compared against?", mode="tree_traversal")
print(len(nodes))
print(nodes[0].text)

Retrieved parent IDs from level 2: ['3df56359-6891-4ce2-96ad-dccdbacc5e6f']
Retrieved 2 from parents at level 2.
Retrieved parent IDs from level 1: ['229049a3-136a-4728-8627-d8c5de8840f1', '889951a1-2ef9-4a39-9a92-8dd1c0a08c88']
Retrieved 4 from parents at level 1.
Retrieved parent IDs from level 0: ['82b1e16c-afa7-4d76-bbc0-c01c6c117aef', '90c4dd8f-ed16-4d80-836d-49ab8eb0eced']
Retrieved 4 from parents at level 0.
4
Specifically, RAPTOR’s F-1 scores are at least 1.8% points higher than DPR and at least 5.3% points
higher than BM25.
Retriever GPT-3 F-1 Match GPT-4 F-1 Match UnifiedQA F-1 Match
Title + Abstract 25.2 22.2 17.5
BM25 46.6 50.2 26.4
DPR 51.3 53.0 32.1
RAPTOR 53.1 55.7 36.6
Table 4: Comparison of accuracies on the QuAL-
ITY dev dataset for two different language mod-
els (GPT-3, UnifiedQA 3B) using various retrieval
methods. RAPTOR outperforms the baselines of
BM25 and DPR by at least 2.0% in accuracy.
Model GPT-3 Acc. UnifiedQA Acc.
BM25 57.3 49.9
DPR 60.4 53.9
RAPTOR 62.4 

In [ ]:
from llama_index.packs.raptor import RaptorRetriever

retriever = RaptorRetriever(
    [],
    embed_model=OpenAIEmbedding(
        model="text-embedding-3-small"
    ), # used for embedding clusters
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1), # used for generating summaries
    vector_store=vector_store, # used for storage
    similarity_top_k=2, # top k for each layer, or overall top-k for collapsed
    mode="tree_traversal", # sets default mode
)

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    raptor_pack.retriever, llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1)
)

response = query_engine.query("What baselines was RAPTOR compared against?")

print(str(response))

RAPTOR was compared against the baselines of BM25 and DPR.
